In [ ]:
## run calculation at diff parames
# make multicore to accelerate
# make fucntion to run in pool

In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import random
from pyfaidx import Fasta

In [2]:
## make dictionary

fn = []
# read parameters
direc = "../data/" 
fn.append(direc+"k3_idx.csv")
fn.append(direc+"k5_idx.csv")
fn.append(direc+"k7_idx.csv") 
fn.append(direc+"k9_idx.csv") 
print(fn[0])


/Users/zrimec/box/Projects-active/Clank4/results/2018_5_14/Data/k3_idx.csv


In [3]:
# make list of dictionaries of lists
di = list()

for i in range(0,4):
    data = pd.read_csv(fn[i],sep=',',header=None)
    data.set_index([0], drop=True, inplace=True)
    
    #di.append(data.to_dict('index'))
    # use via di[1]['AAAAA'][7] - this is problematic
    
    ditmp=[]
    for ii in range(0,data.shape[1]):
        ditmp.append(data.loc[:,ii+1].to_dict())
    di.append(ditmp)
    
# use via di[NN][clust]

dic = di

In [4]:
# nn3c16

dis =[]
for i in range(0,1):
    dis.append(dic[i][2:])
dis1 = dis
    

In [5]:
# nn5c32

dis =[]
for i in range(0,1):
    dis.append(dic[i+1][3:])
dis2 = dis
    

In [6]:
# nn7 c64

dis =[]
for i in range(0,1):
    dis.append(dic[i+2][4:])
dis3=dis
    

In [7]:
# nn9 c128

dis =[]
for i in range(0,1):
    dis.append(dic[i+3][5:])
dis4=dis
    

In [8]:
## sample 10 seqs

random.seed(10)
nums = [x for x in range(0,34)]
random.shuffle(nums)

seqs = Fasta(direc+'ori_nic_37.fasta')

seqs_r10 = list()
seqs_r10_40 = list()
for i in range(0,10):
    seqs_r10.append(seqs[i][:].seq)
    seqs_r10_40.append(seqs[i][111-1:150].seq)

In [9]:
## define all related functions

def colors(zap,di,NN): 
    cz = []
    for i in range(len(zap)-2*NN):
        x = zap[i:i+2*NN+1]
#        print("*",x,kx) 
#        color = di[x][kx]
        color = di[x]
        cz.append(color)
    return cz

def initials(di,colo): 
    intu=[]
    for x in di:
        if di[x] == colo:
            intu.append(x)
    return intu

def reconstructions(zap,di,NN):
    chars = "ACGT"
    intu = initials(di,zap[0])
    nexttu = []
#    print("kx = ",kx)
    for i in range(1,len(zap)):
        #print(i)
        newc = zap[i]
        for tup in intu:
            x = tup[-(2*NN+1):]
            for c in chars:
                y = x[1:]+c 
#                z = di[y]
#                w = z[kx]
                w = di[y]
                if w == newc:
                    newtup = tup+c
                    nexttu.append(newtup) 
        intu = nexttu[:]
#        print(i,len(intu[0]),len(intu))
        nexttu = []
    return intu

def hamming(zap1,zap2): 
    d = 0
    for i in range(len(zap1)):
        if zap1[i] != zap2[i]:
            d += 1
    return d

def ham(li): # ok = p-dist
    n = len(li)
    mat = []
    for i in range(n):
        v =[]
        for j in range(n):
            d = hamming(li[i],li[j])
            v.append(d)
        mat.append(v)
    return mat
# pdist = ham/n

def euclid(c1,c2,m): 
    d = 0
    for i in range(len(c1)):
        for j in range(len(c2)):
            d += m[int(c1[i])-1][int(c2[j])-1]
    return d

#m4 = 1-np.identity(cluster)

def eucl(li,m): # ok
    n = len(li)
    mat = []
    for i in range(n):
        v =[]
        for j in range(n):
            d = euclid(li[i],li[j],m)
            v.append(d) 
        mat.append(v)
    return mat


In [10]:
## function run on single seq instance

def experiment_variants(seq):
    
    n_variants = [] # save all variants, list NN of list clust of list seqs
    pdist = [] # average pdist between reconstructed seqs - might be extra slow
    
    for i in range(0,len(di)): # NN level
        print(i)
        varclust = []
        pdclust = []
        for ii in range(0,len(di[i])): # cluster level
            print('    '+str(ii))
            laz = colors(seq,di[i][ii],i+nn)
            re = reconstructions(laz,di[i][ii],i+nn)
            varclust.append(len(re))
            if len(re)>1:
                mat = np.array(ham(re))
                pdclust.append(np.mean(mat[np.triu_indices(len(mat),1)]))
            else:
                pdclust.append(0)      
        n_variants.append(varclust)
        pdist.append(pdclust)
    
    return [n_variants,pdist]


In [11]:
# run pool
di = dis1
nn = 1
p = Pool(cpu_count()-1)
out1 = p.map(experiment_variants,seqs_r10)

0
0
0
    0
    0
    0
    1
    2
0
    0
    1
    2
0
    0
    1
    2
0
    1
    2
0
    0
    0
    1
    2
0
    0
    1
    2
0
    0
    1
    2
0
    0
    1
    2
    1
    2
    1
    2


In [12]:
out1

[[[[48, 2, 1]], [[4.7659574468085104, 1.0, 0]]],
 [[[384, 8, 1]], [[7.2967798085291555, 1.7142857142857142, 0]]],
 [[[16, 1, 1]], [[3.7333333333333334, 0, 0]]],
 [[[48, 1, 1]], [[7.4893617021276597, 0, 0]]],
 [[[16, 1, 1]], [[4.2666666666666666, 0, 0]]],
 [[[8, 2, 1]], [[2.8571428571428572, 1.0, 0]]],
 [[[64, 4, 1]], [[5.0793650793650791, 1.3333333333333333, 0]]],
 [[[128, 2, 1]], [[7.0551181102362204, 1.0, 0]]],
 [[[768, 1, 1]], [[9.7905258583224679, 0, 0]]],
 [[[16, 1, 1]], [[4.2666666666666666, 0, 0]]]]

In [13]:
outvn = np.ones((4,7,10))*np.NaN
outvm = np.ones((4,7))*np.NaN
outvs = np.ones((4,7))*np.NaN

outpn = np.ones((4,7,10))*np.NaN
outpm = np.ones((4,7))*np.NaN
outps = np.ones((4,7))*np.NaN

for i in range(0,len(out1)):
    #for ii in [0]: # variants / pdist
        #for iii in range(0,2):
    outvn[0,2:5,i] = np.array(out1[i][0][0]) 
    outpn[0,2:5,i] = np.array(out1[i][1][0])   

In [14]:
np.save('temporary_results_var_1.npy',outvn)
np.save('temporary_results_pd_1.npy',outpn)

In [15]:
# run pool
di = dis2
nn = 2
p = Pool(cpu_count()-1)
out2 = p.map(experiment_variants,seqs_r10)

0
0
0
    0
    0
    0
    1
    2
    3
0
    0
    1
    1
    2
    3
0
    0
    2
    3
0
    0
    1
    2
    3
0
    0
    1
    1
    2
    3
0
    0
    1
    2
    3
0
    0
    2
    3
0
    0
    1
    2
    3
    1
    2
    3
    1
    2
    3


In [16]:
out2

[[[[540, 2, 1, 1]], [[6.8905380333951767, 1.0, 0, 0]]],
 [[[384, 432, 1, 1]], [[4.6788511749347261, 4.6774941995359631, 0, 0]]],
 [[[256, 81, 1, 1]], [[3.7647058823529411, 3.6000000000000001, 0, 0]]],
 [[[576, 27, 1, 1]], [[4.563478260869565, 2.7692307692307692, 0, 0]]],
 [[[960, 6, 2, 2]], [[8.5555787278415014, 1.6666666666666667, 1.0, 1.0]]],
 [[[72, 8, 2, 1]], [[3.39906103286385, 1.7142857142857142, 1.0, 0]]],
 [[[256, 12, 1, 1]], [[4.6431372549019612, 3.6969696969696968, 0, 0]]],
 [[[8, 24, 4, 1]], [[1.8571428571428572, 3.5362318840579712, 2.0, 0]]],
 [[[192, 36, 2, 1]], [[4.6910994764397902, 3.9428571428571431, 1.0, 0]]],
 [[[96, 8, 1, 1]], [[4.2105263157894735, 2.2857142857142856, 0, 0]]]]

In [17]:
for i in range(0,len(out2)):
    #for ii in [0]: # variants / pdist
        #for iii in range(0,2):
    outvn[1,3:,i] = np.array(out2[i][0][0])   
    outpn[1,3:,i] = np.array(out2[i][1][0])      

In [18]:
np.save('temporary_results_var_2.npy',outvn)
np.save('temporary_results_pd_2.npy',outpn)

In [19]:
# run pool
di = dis3
nn = 3
p = Pool(cpu_count()-1)
out3 = p.map(experiment_variants,seqs_r10)

0
0
0
    0
    0
    0
    1
    1
    2
0
    0
    1
    2
0
    0
    2
0
    0
    1
    2
0
    0
    1
    2
0
    0
    1
    2
0
    0
    1
    2
0
    0
    1
    2
    1
    2
    1
    2


In [20]:
out3

[[[[216, 40, 1]], [[5.5534883720930228, 3.8564102564102565, 0]]],
 [[[128, 576, 4]],
  [[4.0314960629921259, 5.2869565217391301, 1.3333333333333333]]],
 [[[384, 512, 2]], [[5.5979112271540474, 6.1682974559686885, 1.0]]],
 [[[5120, 48, 2]], [[7.5314709904278176, 3.9716312056737588, 1.0]]],
 [[[17664, 80, 15]],
  [[8.1755310987432175, 4.3848101265822788, 2.1047619047619048]]],
 [[[2304, 6, 4]], [[7.1003039513677813, 1.9333333333333333, 2.0]]],
 [[[16128, 108, 12]],
  [[8.5515506382375612, 3.4766355140186915, 3.8787878787878789]]],
 [[[288, 6, 2]], [[6.2996515679442506, 1.3999999999999999, 2.0]]],
 [[[1536, 48, 4]], [[5.4201954397394134, 3.2340425531914891, 1.0]]],
 [[[38640, 468, 4]],
  [[9.6875395261666402, 5.1612584417723602, 1.3333333333333333]]]]

In [21]:
for i in range(0,len(out2)):
    #for ii in [0]: # variants / pdist
        #for iii in range(0,2):
    outvn[2,4:,i] = np.array(out3[i][0][0])   
    outpn[2,4:,i] = np.array(out3[i][1][0])

In [22]:
np.save('temporary_results_var_3.npy',outvn)
np.save('temporary_results_pd_3.npy',outpn)

In [30]:
# run pool
di = dis4
nn = 4
p = Pool(cpu_count()-1)
out4 = p.map(experiment_variants,seqs_r10)

0
0
    0
0
    0
    0
    1
0
    0
    1
0
    0
    1
0
    0
    1
0
    0
    1
0
    0
    1
0
    0
    1
0
    0
    1
    1
    1


In [26]:
out4

[[[[152, 10]], [[5.5294527710003489, 3.0666666666666669]]],
 [[[32, 48]], [[3.096774193548387, 3.9716312056737588]]],
 [[[6656, 216]], [[8.96871062821476, 5.8604651162790695]]],
 [[[2160, 24]], [[8.6600861166863954, 3.0144927536231885]]],
 [[[160, 10]], [[4.0855345911949685, 1.5555555555555556]]],
 [[[40, 4]], [[3.6923076923076925, 2.0]]],
 [[[4608, 68]], [[9.3909268504449752, 3.5856014047410008]]],
 [[[64, 24]], [[3.4920634920634921, 3.0434782608695654]]],
 [[[72, 8]], [[4.28169014084507, 1.7142857142857142]]],
 [[[152, 4]], [[4.0592540955036602, 1.3333333333333333]]]]

In [28]:
for i in range(0,len(out2)):
    #for ii in [0]: # variants / pdist
        #for iii in range(0,2):
    outvn[3,5:,i] = np.array(out4[i][0][0])   
    outpn[3,5:,i] = np.array(out4[i][1][0])

In [29]:
np.save('temporary_results_var_4.npy',outvn)
np.save('temporary_results_pd_4.npy',outpn)

In [31]:
# calculate averages where possible
outvm = np.ones((4,7))*np.NaN
outvs = np.ones((4,7))*np.NaN
outpm = np.ones((4,7))*np.NaN
outps = np.ones((4,7))*np.NaN

for i in range(0,4):
    for ii in range(0,7):
        if not(np.isnan(outvn[i,ii,0])):
            outvm[i,ii] = np.mean(outvn[i,ii,:])
            outvs[i,ii] = np.std(outvn[i,ii,:])
            outpm[i,ii] = np.mean(outpn[i,ii,:])
            outps[i,ii] = np.std(outpn[i,ii,:])

In [32]:
pd.DataFrame(outvm,index=np.arange(1,5),columns=np.arange(1,8))

,1,2,3,4,5,6,7
1,NaN,NaN,149.6,2.3,1.0,NaN,NaN
2,NaN,NaN,NaN,334.0,63.6,1.6,1.1
3,NaN,NaN,NaN,NaN,8240.8,189.2,5.0
4,NaN,NaN,NaN,NaN,NaN,1409.6,41.6


In [33]:
pd.DataFrame(outvs,index=np.arange(1,5),columns=np.arange(1,8))

,1,2,3,4,5,6,7
1,NaN,NaN,232.249521,2.100000,0.000000,NaN,NaN
2,NaN,NaN,NaN,275.548907,124.759128,0.916515,0.300000
3,NaN,NaN,NaN,NaN,11917.409893,218.934145,4.427189
4,NaN,NaN,NaN,NaN,NaN,2245.341631,61.421820


In [34]:
pd.DataFrame(outpm,index=np.arange(1,5),columns=np.arange(1,8))

,1,2,3,4,5,6,7
1,NaN,NaN,5.660092,0.604762,0.000000,NaN,NaN
2,NaN,NaN,NaN,4.725412,2.888945,0.500000,0.100000
3,NaN,NaN,NaN,NaN,6.794914,3.887338,1.565022
4,NaN,NaN,NaN,NaN,NaN,5.525680,2.914551


In [35]:
pd.DataFrame(outps,index=np.arange(1,5),columns=np.arange(1,8))

,1,2,3,4,5,6,7
1,NaN,NaN,2.04011,0.637117,0.000000,NaN,NaN
2,NaN,NaN,NaN,1.748386,1.127264,0.670820,0.300000
3,NaN,NaN,NaN,NaN,1.635252,1.401448,0.976609
4,NaN,NaN,NaN,NaN,NaN,2.361937,1.300771


In [37]:
np.savez('variants_220bp-18_5_18.npz',outvn,outvm,outvs,outpn,outpm,outps)
